In [1]:
import pandas as pd
import re
import time

In [3]:
start = time.time()
chemistry_post = pd.read_csv("../data/csv/chemistry_post.csv", lineterminator='\n')
physics_post = pd.read_csv("../data/csv/physics_post.csv", lineterminator='\n')
biology_post = pd.read_csv("../data/csv/biology_post.csv", lineterminator='\n')
end = time.time()
print((end-start)/60.0)

/Users/shuang/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0.10926812887191772


In [8]:
biology_post.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48853 entries, 0 to 48852
Data columns (total 22 columns):
AcceptedAnswerId         9894 non-null float64
AnswerCount              22002 non-null float64
Body                     48853 non-null object
ClosedDate               2318 non-null object
CommentCount             48853 non-null int64
CommunityOwnedDate       117 non-null object
CreationDate             48853 non-null object
FavoriteCount            7858 non-null float64
Id                       48853 non-null int64
LastActivityDate         48853 non-null object
LastEditDate             25464 non-null object
LastEditorDisplayName    562 non-null object
LastEditorUserId         24935 non-null float64
OwnerDisplayName         1187 non-null object
OwnerUserId              47835 non-null float64
ParentId                 25723 non-null float64
PostTypeId               48853 non-null int64
Score                    48853 non-null int64
Tags                     22002 non-null object
Titl

In [10]:
biology_post.head()

,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,...,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount,Equation
0,59.0,1.0,"<p>In prokaryotic translation, how critical fo...",NaN,0,NaN,2011-12-14T21:17:44.370,7.0,1,2012-02-20T17:42:02.997,...,338.0,NaN,28.0,NaN,1,24,<ribosome><binding-sites><translation><synthet...,What is the criticality of the ribosome bindin...,7061.0,None
1,12.0,3.0,<p>Does anyone have any suggestions to prevent...,NaN,4,NaN,2011-12-14T21:17:48.270,1.0,2,2012-02-16T11:10:57.530,...,338.0,NaN,12.0,NaN,1,21,<rna><biochemistry>,How is RNAse contamination in RNA based experi...,1375.0,None
2,31.0,1.0,<p>Tortora writes in <em>Principles of Anatomy...,NaN,0,NaN,2011-12-14T21:17:50.423,NaN,3,2012-01-22T10:34:58.940,...,66.0,user24,NaN,NaN,1,22,<immunology><cell-biology><hematology>,Are lymphocyte sizes clustered in two groups?,781.0,None
3,105.0,3.0,<p>Various people in our lab will prepare a li...,NaN,0,NaN,2011-12-14T21:19:43.527,2.0,4,2018-10-03T13:40:21.967,...,25.0,NaN,25.0,NaN,1,28,<cell-culture>,How long does antibiotic-dosed LB maintain goo...,18983.0,None
4,24.0,3.0,<p>Are there any cases in which the splicing m...,NaN,3,NaN,2011-12-14T21:22:40.450,3.0,5,2012-09-13T08:36:24.423,...,43.0,NaN,43.0,NaN,1,17,<splicing><mrna><spliceosome><introns><exons>,Is exon order always preserved in splicing?,2102.0,None


In [5]:
def regrex_processing_post(subject):
    equation = []
    pattern = re.compile('.*?\$(.+?)\$')
    subject['Body'] = subject['Body'].astype(str)
    for text in subject.Body:
        try:
            if "$" in text:
                eq = re.findall(pattern, text)
                for i in range(len(eq)):
                    eq[i] = eq[i].replace("$", "")
                equation.append(eq)
            else:
                equation.append(None)
        except:
            print('An error occured with ' + text)
    subject['Equation'] = equation
    new_subject = subject[subject['Equation'].notnull()].reset_index(drop = True)
    return new_subject

In [6]:
subject_string = ['chemistry', 'physics', 'biology']
post_df= [chemistry_post, physics_post, biology_post]

In [7]:
for i in range(len(subject_string)):
    start = time.time()
    print("start processing posts for " + subject_string[i])
    subject_post = regrex_processing_post(post_df[i])
    end = time.time()
    print((end-start)/60.0)
    
    subject_post['Label'] = subject_string[i]
    
    start = time.time()
    print("start filtering posts for " + subject_string[i])
    post_rows = []
    _ = subject_post.apply(lambda row: [post_rows.append([eqn, row["Label"], row["Id"], row["Tags"]]) for eqn in row.Equation], axis=1)
    subject_post_filtered = pd.DataFrame(post_rows, columns=["Equation", "Label", "Id", "Tags"])
    end = time.time()
    print(len(subject_post_filtered.index))
    print((end-start)/60.0)
    
    start = time.time()
    print("start saving posts for " + subject_string[i])
    subject_post_filtered.to_csv("../data/equation/" + subject_string[i] + "_post_eq_filtered_id.csv", index = False)
    end = time.time()
    print((end-start)/60.0)

start processing posts for chemistry
0.5218459010124207
start filtering posts for chemistry
199528
0.05985979636510213
start saving posts for chemistry
0.007663249969482422
start processing posts for physics
3.1043565193812053
start filtering posts for physics
1347287
0.39530924558639524
start saving posts for physics
0.0542757511138916
start processing posts for biology
0.09078939755757649
start filtering posts for biology
12571
0.005964132150014242
start saving posts for biology
0.0006248990694681804
